# AWS Strands Agents SDK

Strands is AWS's framework for building production-ready AI agents that can:
- Use multiple AI models (Bedrock, OpenAI, etc.)
- Execute tools and take actions
- Coordinate multiple agents in workflows
- Provide enterprise-grade observability

What we'll cover:
1. Installing and basic setup
2. Creating your first agent
3. Debug logging and observability
4. Switching between models
5. Creating custom tools
6. Using built-in tools
7. Building multi-agent graphs

To install the sdk run pip install `strands-agents`

In [ ]:
pip install -U strands-agents strands-agents-tools

Lets create our first Agent...

In [ ]:
from strands import Agent

# Create an agent with default settings
agent = Agent()

# Ask the agent a question
agent("What is AWS Strands Agents")

### Debug Logs

After running an agent, you can understand what happened during execution through traces and metrics.  Every agent invocation returns an `AgentResult` object with observability data.

To enable debug logs in our agent, configure the `strands` logger:

In [ ]:
import logging
from strands import Agent

# Enable Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG)

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent()

agent("Hi whats your name?")

### Model Providers

Identifying a configured model provider

In [ ]:
print(agent.model.config)

If we want to specify an alternate model we can pass the `model ID` string directly, i'll specify Claude 3 Haiku `Cross Region Inference` profile id

In [ ]:
agent = Agent(model="us.anthropic.claude-3-haiku-20240307-v1:0")

Lets do a test query to confirm our model change occurred

In [ ]:
agent("hi whats your name")

What if you want more control of the model you specify i.e. defining `inference parameters`, you would create a model provider instance...

In [2]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
    region_name="us-east-1",
    temperatur=0.3
)

agent = Agent(model=bedrock_model)

### Creating a Custom Tool and associating it with your Agent

In [5]:
from strands import Agent, tool

@tool
def greet_user(name: str, language: str = "English") -> str:
    """
    Generate a personalized greeting in different languages.
    
    Args:
        name: The person's name to greet
        language: The language for the greeting (English, Spanish, French, Japanese)
    
    Returns:
        A personalized greeting message
    """
    greetings = {
        "English": f"Hello {name}! How are you today?",
        "Spanish": f"¡Hola {name}! ¿Cómo estás?",
        "French": f"Bonjour {name}! Comment allez-vous?",
        "Japanese": f"こんにちは {name}さん! お元気ですか？"
    }
    return greetings.get(language, f"Hello {name}!")

agent = Agent(model="us.anthropic.claude-3-haiku-20240307-v1:0",tools=[greet_user])

In [ ]:
# Natural language invocation - agent decides when to use the tool
result = agent("Please greet Noel in Spanish")
print(result)

In [ ]:
# Direct method call
result = agent.tool.greet_user(name="Noel", language="French")
print(result)

### Built in tools

To work with the built in tools ensure you performed a `pip install strands-agents-tools`, Note Strands offers numerous built-in tools and you can also create your own custom tools, lets show an example of using a built in strands agent tool `calculator`

In [3]:
# Lets turn off DEBUG level logging
import logging

# Set to INFO level - only shows INFO, WARNING, ERROR, CRITICAL
logging.getLogger("strands").setLevel(logging.INFO)

logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

In [ ]:
from strands import Agent
from strands_tools import calculator

# Create agent with calculator tool
agent = Agent(
    tools=[calculator],
    model=bedrock_model
    #callback_handler=None  # Disable default callback handler
)

# Basic arithmetic examples
result = agent("What is 25 * 47?")
print(result)

### What if you wanted to define a `Graph` to execute a sequence of steps

You can use the Strands built-in `graph` tool to create and manage multi-agent systems using Strands SDK Graph implementation

In [ ]:
from strands import Agent
from strands_tools import graph, python_repl
import os

# Bypass tool consent
os.environ["BYPASS_TOOL_CONSENT"] = "true"

# Create the main agent with the necessary tools
agent = Agent(
    model="us.anthropic.claude-3-haiku-20240307-v1:0",
    tools=[graph, python_repl],
    callback_handler=None
)

# Step 1: Create the graph with very explicit instructions
result = agent("""
Use the graph tool to create a graph. Call it with these exact parameters:

action: "create"
graph_id: "code_pipeline"
topology: {
    "nodes": [
        {
            "id": "code_generator",
            "role": "coder",
            "system_prompt": "Generate Python code for the given task. Output only the code, no explanations or markdown."
        },
        {
            "id": "code_executor",
            "role": "executor",
            "system_prompt": "Execute the provided Python code using the python_repl tool and report the results.",
            "tools": ["python_repl"]
        }
    ],
    "edges": [
        {"from": "code_generator", "to": "code_executor"}
    ],
    "entry_points": ["code_generator"]
}

Make sure to pass all these parameters exactly as shown.
""")

print("Graph creation result:", result)

# Check if graph was created successfully
list_result = agent("Use the graph tool to list all graphs (action='list')")
print("\nAvailable graphs:", list_result)

# Step 2: Execute a task through the graph
result = agent("""
Use the graph tool with these parameters:
- action: "execute"
- graph_id: "code_pipeline"
- task: "Create a Python function that generates the Fibonacci sequence up to n terms, then call it with n=10 and print the result"
""")

print("\nExecution result:", result)

### Human-in-the-Loop

An important features for production AI systems is the ability to pause and get human approval or input before taking critical actions.  Strands provides the `handoff_to_user` tool for this.  Let's execute this from a python file in the command line so it can handle our input correctly.

```python

from strands import Agent
from strands_tools import handoff_to_user, file_write, shell
import os

os.environ["BYPASS_TOOL_CONSENT"] = "true"  # For demo purposes only

# Create an agent with human handoff capability
agent = Agent(
    model="us.anthropic.claude-3-haiku-20240307-v1:0",
    tools=[handoff_to_user, file_write, shell],
    system_prompt="""You are a helpful assistant that asks for human 
    confirmation before performing potentially dangerous operations."""
)

# Example 1: Getting user approval before file operations
result = agent("""
I need to delete an old log file named test.log located at /Users/nsinghsr/KiroProjects/Strands and 
create a new log file named newtest.log located in the same directory.  I want the content in the new log file
to state "test from noel aug 13th".  Please handle this safely.
""")
```